<a href="https://colab.research.google.com/github/yelim24/kaggle_CommonLit_summary/blob/main/%5Breview%5Dwinning_solution_commonlit_readibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 현재는 문제 없지만 추후에 문제가 있을 요소들 경고 끄기
import warnings
warnings.filterwarnings("ignore")

# DATA

In [ ]:
import pandas as pd

In [ ]:
train_pro = pd.read_csv("/content/drive/MyDrive/data_yelim/commonlit-evaluate-student-summaries/prompts_train.csv")
train_pro

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [ ]:
train_sum = pd.read_csv("/content/drive/MyDrive/data_yelim/commonlit-evaluate-student-summaries/summaries_train.csv")
train_sum.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


# Data Preparations

In [ ]:
# os 모듈 호출하기 위해
import os
# 반복문에서 진행률을 '진행바'로 확인하기 위해
import tqdm
# 교차검증을 하기 위해(StratifiedKFold : 불균형한 분포도를 가진 레이블 데이터 집합을 위한 KFold 방식)
from sklearn.model_selection import StratifiedKFold

In [ ]:
# prompt_id를 기준으로 train_pro + train_sum
train = train_pro.merge(train_sum , on = "prompt_id")
train.head()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886


In [ ]:
def bootstrap(n_bags):

    # ==== Create bins and fold
    # 학습 데이터의 content 값을 6등분해서 train["bins"]에 저장
    train["bins"] = pd.cut(train["content"] , n_bags , labels = [i for i in range(n_bags)])
    train["fold"] = 0

    # ==== Stratified K Fold Cross Validation
    # fold를 5개로 나눔
    skf = StratifiedKFold(n_splits = 5 , shuffle = True)
    # (feature, label) label분포를 동일하게 나눔
    # skf.split(train["student_id"], train["bins"])로 작성해도 될듯
    gen_skf = skf.split(train["student_id"], y = train["bins"])

    # ==== Update with each iteration
    # train_idx(학습용), val_idx(검증용) 당연히 train_idx+val_idx 수는 5번 다 동일
    for index , (train_idx , val_idx) in enumerate(gen_skf):
        # train["fold"]에는 해당 행이 몇 번째 검증 데이터로 이용되는지 저장됨
        train["fold"][val_idx] = index

    # ==== Frame creation and saving
    # 디렉터리 생성
    os.makedirs("/kaggle/working/Pseudo Dir/K Folds/Folds/Train")
    os.makedirs("/kaggle/working/Pseudo Dir/K Folds/Val")

    # 진행바를 통해서 진행률을 보여줌, n_bags -> 5로 수정
    for fold in tqdm.tqdm(range(5) , total = 5):
        # 해당 순서(fold)에서 학습 데이터로 사용되는 것만 저장(train에서 train["fold"]가 fold가 아닌 것만 저장)
        sample_train = train[train["fold"] != fold]
        # 해당 순서(fold)에서 검증 데이터로 사용되는 것만 저장
        sample_val = train[train["fold"] == fold]
        # print(sample_train["bins"].value_counts())
        sample_train.to_csv("/kaggle/working/Pseudo Dir/K Folds/Folds/Train/Train_" + str(fold) + ".csv")
        sample_val.to_csv("/kaggle/working/Pseudo Dir/K Folds/Val/Val_" + str(fold) + ".csv")

In [ ]:
# rm -rf ../kaggle

In [ ]:
bootstrap(n_bags = 6)

100%|██████████| 5/5 [00:11<00:00,  2.37s/it]


# Tokenization

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
import numpy as np

In [ ]:
# AutoTokenizer.from_pretrained -> 모델과 매치되는 tokenizer를 불러옴
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer

RobertaTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [ ]:
# return_tensors : "tf" 면 tf.constant, "pt" 면 torch.Tensor, "np" 면 np.ndarray 로 결과를 리턴
# attention_mask -> padding token은 무시할 수 있도록 1/0으로 구분해서 값을 지정
tokenizer("Your dreams are shining in the darkness. There is a intoxication in your eyes, You are in my dreams, in the answers, in the questions, Every day, I steal you in my thoughts" , return_tensors = "np")["input_ids"]

{'input_ids': array([[    0, 12861,  7416,    32, 21003,    11,     5, 15073,     4,
          345,    16,    10, 34205,    11,   110,  2473,     6,   370,
           32,    11,   127,  7416,     6,    11,     5,  5274,     6,
           11,     5,  1142,     6,  4337,   183,     6,    38,  8052,
           47,    11,   127,  4312,     2]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
os.makedirs("/kaggle/working/Pseudo Dir/Embeds")

In [ ]:
# 6 -> 5로 수정
for fold in range(5):
    train = pd.read_csv("/kaggle/working/Pseudo Dir/K Folds/Folds/Train/Train_" + str(fold) + ".csv")
    # train.shape[0] -> 행(data) 개수 / dtype -> 데이터 타입 지정
    # train_1.csv 기준 (5732,) 크기의 빈 array 생성
    tokens = np.empty(shape = train.shape[0] , dtype = np.ndarray) #tokens.shape -> (5732,)

    for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
        stri = ""
        # 이걸 왜 다 더하는지? 걍 다 때려박으려고 더하는 것인지?
        for column in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]:
            stri += str(train[column][index])

        # padding='max_length' 추가
        token = tokenizer(stri, padding='max_length', return_tensors = "np")["input_ids"]
        tokens[index] = token

    # array 파일로 저장
    np.save("/kaggle/working/Pseudo Dir/Embeds/Embeds_" + str(fold) , tokens)

100%|██████████| 5732/5732 [00:22<00:00, 255.79it/s]


# DataLoader

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

In [ ]:
class load_train(Dataset):

    def __init__(self , x_train_dir , y_train_dir , target_type = "content"):

        # comment 코드 추가
        self.x_train_dir = x_train_dir # token
        self.y_train_dir = y_train_dir # 학습 데이터

        # 저장해놨던 array 불러옴
        self.x_train = np.load(self.x_train_dir, allow_pickle = True)

        y = pd.read_csv(self.y_train_dir)

        if target_type == "content":self.y_train = y["content"]
        if target_type == "wording":self.y_train = y["wording"]

    def __len__(self) : return self.x_train.shape[0]

    # a = load_train(~~)
    # a[1] 과 같이 호출
    # 클래스의 인덱스에 접근할 때 자동으로 호출되는 메서드
    def __getitem__(self , index):
        # 데이터는 array 형식 입력, 텐서타입 LongTensor으로 지정
        # dtype = torch.long -> 이거 없어도 돌아가고 왜 있는지는 모르겠음
        r_embed = torch.tensor(self.x_train[index], dtype = torch.long)
        r_target = torch.tensor(self.y_train[index], dtype = torch.long)

        return r_embed , r_target

In [ ]:
# train = load_train(x_train_dir = "/kaggle/working/Pseudo Dir/Embeds/Embeds_0.npy" ,
#                   y_train_dir = "/kaggle/working/Pseudo Dir/K Folds/Folds/Train/Train_5.csv")

# train_dataset = torch.utils.data.DataLoader(train , shuffle = True , batch_size = 1)

# Model Setup

In [ ]:
# AutoConfig -> Model ID만 있으면 맞는 config(학습에 필요한 파라미터들을 정의해주는 파일)를 불러옴
# AutoModelForSequenceClassification -> pretrained model 불러오기
from transformers import AutoConfig , AutoModelForSequenceClassification
import torch

In [ ]:
# 추가
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 2.0.1+cu118  Device: cuda


In [ ]:
# AlBERT XX Large V2
# Takes around  30−36 Minutes, 5−6 Minutes for each fold
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels = 1)
# 추가
model.to(device)

In [ ]:
# DebERTa Large
# Takes around 2 Hours, 20 Minutes for each fold
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-large", num_labels = 1)

In [ ]:
# RoBERTa Large
# Takes around 1.2 Hours, 14 Minutes for each fold
model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels = 1)

In [ ]:
# ELECTRA
# Takse around 12 Minutes, 2 Minutes for each fold

# Training Arguments

In [ ]:
# loss 함수 제공 클래스(신경망 계층(layer)과 거의 비슷한 Module 의 집합을 정의)
import torch.nn as nn

# Mean Squared Error 평균 제곱 오차
loss_func = nn.MSELoss()
# AdamW 옵티마이저
# (AdamW는 Adam 옵티마이저의 변형으로, 가중치 감쇠(weight decay)를 적용하는 것이 특징입니다.
# 가중치 감쇠는 모델의 가중치를 감소시킴으로써 모델의 복잡성을 제어하고,
# 오버피팅(overfitting)을 완화하는 효과가 있습니다)
# model.parameters() -> PyTorch 모델의 학습 가능한 파라미터들을 반환하는 메소드(각 layer에 저장된 weight와 bias 값들)
optimizer = torch.optim.AdamW(model.parameters() , lr  = 9e-6)

# Training Loop

각 학습 단계(training loop)에서 모델은 (배치(batch)로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정

In [ ]:
# 테스트 코드
x_train_dir = "/kaggle/working/Pseudo Dir/Embeds/Embeds_0.npy"
y_train_dir = "/kaggle/working/Pseudo Dir/K Folds/Folds/Train/Train_0.csv"

train = load_train(x_train_dir = x_train_dir ,
                   y_train_dir = y_train_dir)

train_d = torch.utils.data.DataLoader(train , shuffle = True , batch_size = 1)

for i, (data, target) in enumerate(train_d):
    print(i, data.shape, target.shape, target)
    print(data[0].shape)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# Weight and Biases 사이트와 연동해 visualization 결과를 확인 가능
# https://89douner.tistory.com/313
# wandb.watch(model , loss_func)

# 6 -> 5로 수정
for fold in range(5):
    #  GPU 캐시 데이터 삭제 (GPU 메모리가 부족할 때 확인, 학습 전 사용)
    torch.cuda.empty_cache()

    x_train_dir = "/kaggle/working/Pseudo Dir/Embeds/Embeds_" + str(fold) + ".npy"
    y_train_dir = "/kaggle/working/Pseudo Dir/K Folds/Folds/Train/Train_" + str(fold) + ".csv"

    # train.x_train, train.y_train
    train = load_train(x_train_dir = x_train_dir ,
                       y_train_dir = y_train_dir)

    # 데이터셋(dataset)을 읽어와서 배치(batch) 단위로 데이터를 불러옴
    # input으로 전체 x (input feature)와 y (label)을 tensor로 넣어주면 됨
    # shuffle -> 매 에폭(epoch)마다 데이터셋이 섞임
    # https://hi-guten-tag.tistory.com/345
    train_d = torch.utils.data.DataLoader(train , shuffle = True , batch_size = 1)

    torch.cuda.empty_cache()

    # x = data, y = label
    for x , y in tqdm(train_d , total = len(train_d), leave=True):

        torch.cuda.empty_cache()

        # maximum sequence length for this model (512)
        x = x[0][: , :512] # torch.Size([1, 869])
        # clip()=clamp() 텐서x의 최대 최소 범위 고정 왜 하는지 모르겠음..
        # to("cuda") -> to(device)로 수정 / GPU에 모델을 할당
        x = torch.clip(x , min = 0 , max = 29999).to(device)
        y = y.to(device)

        # 예측 오류 계산
        # logits -> 모르겠음;; 지성지성;;
        pred = model(x).logits
        loss = loss_func(pred.to(torch.float32), y.to(torch.float32))

        # wandb.log({"loss": loss})

        # 추가
        # Pytorch에서는 gradients값들을 추후 backward를 해줄 때 계속 더해주기 때문에
        # 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야함
        optimizer.zero_grad()
        # 예측 손실(prediction loss)을 역전파
        loss.backward()
        # 역전파 단계에서 수집된 변화도로 매개변수를 조정(학습이 이루어짐)
        optimizer.step()

        # torch.cuda.empty_cache()


  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

# Results

In [ ]:
from IPython.display import IFrame